In [ ]:
!pip install python-Levenshtein tha datasets khmerspell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [ ]:
import numpy as np
from datasets import load_dataset, Audio
from Levenshtein import distance
from tqdm import tqdm
from IPython.display import Audio as IPAudio
from prettytable import PrettyTable
import re
from tha.decimals import processor
from khmerspell import khnormal

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('hg-main'))

In [ ]:
dataset = load_dataset('PhanithLIM/gfleurs-evaluation', split='test')
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/578 [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/154M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/334 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'text', 'mms', 'whisper-base-aug-20-april-lightning-v1-pipeline', 'whisper-base-khmer-pipeline', 'whisper-base-aug-11-may-lightning-v1'],
    num_rows: 334
})


In [ ]:
def remove_symbols(example):
    symbols_to_remove = [
        '(', ')', '[', ']', '{', '}', '<', '>',
        '“', '”', '‘', '’', '«', '»', ',', '?',
        '「', '」', '『', '』', '▁', '-', ' ', "%", '.',
        '៖', '។', '៛', '៕', '!', '​', '–', 'ៗ', '�', ''
    ]
    for col, value in example.items():
        if isinstance(value, str):  # Ensure the value is a string before processing
            for symbol in symbols_to_remove:
                value = value.replace(symbol, '')
            example[col] = value
    return example

def convert_num2text(example):
    for col, value in example.items():
        if isinstance(value, str):  # Ensure the value is a string before applying regex
            value = re.sub(r'[0-9,]+(?:\.[0-9,]+)?', lambda match: processor(match.group(0)), value)
            value = re.sub(r'[០-៩,]+(?:\.[០-៩,]+)?', lambda match: processor(match.group(0)), value)
            example[col] = value.replace('▁', ' ')
    return example

def filter_english_rows(example):
    for col, value in example.items():
        if isinstance(value, str) and bool(re.search(r'[a-zA-Z]', value)):
            return False
    return True

def is_nonempty_row(example):
    for col, value in example.items():
        if isinstance(value, str) and value.strip():
            return True
    return False

def filter_long_audio(example):
    audio_array = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]
    duration = len(audio_array) / sampling_rate
    return duration <= 30  # keep if less than 30 sec

In [ ]:
dataset2 = dataset.filter(filter_long_audio)
dataset2 = dataset2.map(convert_num2text)
dataset2 = dataset2.map(remove_symbols)
dataset2 = dataset2.filter(filter_english_rows)
dataset2 = dataset2.filter(is_nonempty_row)

Filter:   0%|          | 0/334 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

Filter:   0%|          | 0/329 [00:00<?, ? examples/s]

Filter:   0%|          | 0/310 [00:00<?, ? examples/s]

In [ ]:
dataset.column_names

['audio',
 'text',
 'mms',
 'whisper-base-aug-20-april-lightning-v1-pipeline',
 'whisper-base-khmer-pipeline',
 'whisper-base-aug-11-may-lightning-v1']

In [ ]:
pt = PrettyTable()
pt.field_names = ['model', 'cer (%)']
pt.align['model'] = 'l'
pt.align['cer (%)'] = 'l'

ref_col = 'text'
columns_kept = [col for col in dataset2.column_names if col != 'audio' and col != ref_col]
columns_kept.insert(0, ref_col)

# Loop over each model column (skip the reference column)
for col in columns_kept[1:]:
    count_chars = 0
    count_errors = 0

    for example in dataset2:
        ref_text = khnormal(str(example[ref_col]))
        hyp_text = khnormal(str(example[col]))
        if not ref_text.strip() or not hyp_text.strip():
            continue
        err = distance(ref_text, hyp_text)
        if err < 100:
            count_chars += len(ref_text)
            count_errors += err

    cer = (count_errors / count_chars) * 100 if count_chars > 0 else 0
    pt.add_row([col, f"{cer:.3f}"])
# Print the results table
print('Model Performance on Khmer Test Set')
print(pt)

Model Performance on Khmer Test Set
+-------------------------------------------------+---------+
| model                                           | cer (%) |
+-------------------------------------------------+---------+
| mms                                             | 15.475  |
| whisper-base-aug-20-april-lightning-v1-pipeline | 15.228  |
| whisper-base-khmer-pipeline                     | 14.833  |
| whisper-base-aug-11-may-lightning-v1            | 14.554  |
+-------------------------------------------------+---------+
